
# <span style="color:DarkSeaGreen">JumpStart Lab 1</span>

This lab does the following:

- Provision a model via Jumpstart
- Create a JumpStart endpoint
- Interacts with the model



# <span style="color:DarkSeaGreen">Prepare Your Environment</span>
### Requirements for this Jupyter Notebook Lab if running in VSCode or equivalent local IDE
##### Note these are macOS specific
- Credentials
  - You need credentials to your AWS account to execute this Jupyter Lab if running locally from your laptop
    - Locally: Credentials and therefore permissions asscociated with the IAM user (with CLI access enabled) are provided by AWS configure connection to your AWS account
    - Cloud: Permissions provided via logged in user
- Installers:
  - Pip
    - Python libraries
    - Works inside Python envs
  - homebrew (brew) (mac)
    - System software, tools, and dependencies
    - Works at OS level

- Run the commands of the cell below in a terminal window to create a virtual environment if you need one
  - Note check your Python version first, then if ok, copy the rest and run in terminal window
  - Note if you copy and paste the multiple lines and run as one you will get zsh: command not found: # errors because of the comments, but you can ignore
  - Remember to restart the kernel to pick up the new venv
  - The venv can be deleted via the last cell in this notebook iof no longer needed
- If you already have a virtual environment, then just activate it as shown in the second cell below
  - Venv (can be created below) used by this notebook is *venv-jumpstart-lab1*

In [ ]:
# Check your credentials (AWS identity) to confirm you are using the right credentials, can also run in a terminal window (remove the !)
!aws sts get-caller-identity

In [ ]:
### STOP ###
### IF USING THIS NOTEBOOK IN A SAGEMAKER JUPYTER NOTEBOOK INSTANCE, THEN SKIP TO THE NEXT CELL ###
### OTHERWISE, IF USING VSCODE OR EQUIVALENT LOCAL IDE, THEN CONTINUE BELOW ###
### This script is for setting up your environment for the JumpStart Lab 1 ###
# do you need to upgrade python first?
python3 --version

### STOP ###
### DO YOU NEED TO UPGRADE PYTHON ###
# upgrade to the latest version of python if required
brew install python
# restart vscode to pickup new version of python
python3 --version

### STOP ###
### OK IF YOU HAVE THE CORRECT VERSION OF PYTHON, CONTINUE ###
# create a virtual environment
python3 -m venv venv-jumpstart-lab1
# activate the virtual environment
source venv-jumpstart-lab1/bin/activate
### COPY TO HERE ONLY IF RUNNING AS ONE COPY AND PASTE ###

### STOP ###
### MAKE SURE ABOVE VENV GETS ACTIVATED BEFORE RUNNING THE REST ###
# upgrade pip
pip install --upgrade pip
# jupyter kernel support
pip install ipykernel
# add the virtual environment to jupyter
python  -m ipykernel install --user --name=venv-jumpstart-lab1 --display-name "Python (venv-jumpstart-lab1)"
# install the required packages
pip install -r Documents/github/labs/sagemaker/jumpstart/requirements_lab1.txt
# verify the installation
pip list

### RESTART VSCODE TO PICKUP THE NEW VENV ###

In [ ]:
### STOP ###
### This command is for activating an environment that already exists, its for use in a terminal window if you need it ###
source venv-jumpstart-lab1/bin/activate
pip list

# use pip freeze if you prefer for requirements.txt freiendly format
### ALSO MAKE SURE YOU SELECT IT AS YOUR KERNEL FOR THIS JUPYTER NOTEBOOK ###

In [ ]:
### STOP ###
### IF USING THIS NOTEBOOK IN A SAGEMAKER JUPYTER NOTEBOOK INSTANCE, THEN EXECUTE THIS CELL ###
!pip install --upgrade pip
!pip install --upgrade sagemaker

# Lab 1 Starts Here!

# <span style="color:DarkSeaGreen">Setup</span>

In [ ]:
# region
myRegion='ap-southeast-2'

# iam
myRoleSageMakerExecution="doit-jumpstart-sagemaker-execution-role"
myRoleSageMakerExecutionARN='RETRIEVED FROM ROLE BELOW'

print ('Done! Move to the next cell ->')

In [ ]:
import json
import boto3
import sagemaker
from certifi import where

# Configure boto3 to use certifi's certificates - helps avoid SSL errors if your system’s certificate store is out of date or missing root certs
sts_client = boto3.client('sts', verify=where())
myAccountNumber = sts_client.get_caller_identity()["Account"]
print(myAccountNumber)
print(sts_client.get_caller_identity()["Arn"])

# create clients we can use later
# iam
iam = boto3.client('iam', region_name=myRegion, verify=where())

# sagemaker
sagemaker = boto3.client('sagemaker', region_name=myRegion, verify=where())

print ('Done! Move to the next cell ->')

In [ ]:
# define tags added to all services we create
myTags = [
    {"Key": "env", "Value": "non_prod"},
    {"Key": "owner", "Value": "doit-jumpstart"},
    {"Key": "project", "Value": "lab1"},
    {"Key": "author", "Value": "simon"},
]
myTagsDct = {
    "env": "non_prod",
    "owner": "doit-jumpstart",
    "project": "lab1",
    "author": "simon",
}

print ('Done! Move to the next cell ->')

# <span style="color:DarkSeaGreen">IAM</span>

In [ ]:
def getSageMakerExecutionRole():
    """
    Creates a role required for SageMaker to run jobs on your behalf
    Only needed if this is being run in a local IDE, not needed if in SageMaker Studio or SageMaker Notebook Instance

    Args:
        None

    Returns:
        An IAM execution role ARN
    """

    # trust policy for the role
    roleTrust = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "sagemaker.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    # check if the role exists
    try:
        role = iam.get_role(RoleName=myRoleSageMakerExecution)
        print("Role already exists. Using the existing role.")
        return role['Role']['Arn']
    except iam.exceptions.NoSuchEntityException:
        print("Role does not exist. Creating a new role.")
        
    # create execution role for sagemaker - allows SageMaker notebook instances, training jobs, and models to access S3, ECR, and CloudWatch on your behalf
    # this role is only created if we are running this notebook in a local ide, if we are in a jupyterlab in sagemaker studio, we dont need it as already created and available
    role = iam.create_role(
        RoleName=myRoleSageMakerExecution,
        AssumeRolePolicyDocument=json.dumps(roleTrust),
        Description="Service excution role for sagemaker ai use including inside jupyter notebooks",
        Tags=[
            *myTags,
        ],
    )

    # attach managed policy to the role AmazonSageMakerFullAccess
    iam.attach_role_policy(
        RoleName=myRoleSageMakerExecution,
        PolicyArn="arn:aws:iam::aws:policy/AmazonSageMakerFullAccess"
    )

    return role['Role']['Arn']

# <span style="color:DarkSeaGreen">Get Execution Role</span>
- SageMaker requires an execution role to assume on your behalf

In [ ]:
from sagemaker import get_execution_role
from sagemaker.jumpstart.model import JumpStartModel
from sagemaker.session import Session

try:
    # if this is being run in a SageMaker AI JupyterLab Notebook
    myRoleSageMakerExecutionARN = get_execution_role()
except:
    # if this is being run in a local IDE - we need to create our own role
    myRoleSageMakerExecutionARN = getSageMakerExecutionRole()

sess = Session()

print(myRoleSageMakerExecutionARN)
print(sess)

print ('Done! Move to the next cell ->')

# <span style="color:DarkSeaGreen">Provision a JumpStart Model</span>
- Provision a model via Jumpstart
- If you prefer, you can also do this via the JumpStart console, but you will have to bring in the endpoint that you create to continue with this code

In [19]:
# the model we want to provision
# jump into the console, click on JumpStart, find a model you like and copy the model id from the details page
# Qwen2.5 Coder 32B Instruct - requires a large instance to run, eg ml.g6.48xlarge - DO NOT LEAVE THIS RUNNING LONGER THAN YOU NEED TO
# see https://aws.amazon.com/sagemaker/ai/pricing/
model_id, model_version = "huggingface-llm-qwen2-5-coder-32b-instruct", "*"
# if your selected model is gated, you will need to set accept_eula to True to accept the model end-user license agreement (EULA)
accept_eula = True

# https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.jumpstart.model.JumpStartModel
model = JumpStartModel(model_id=model_id, model_version=model_version, instance_type="ml.g6.48xlarge", role=myRoleSageMakerExecutionARN, sagemaker_session=sess)

# deploy the model to an endpoint
# this will take a while
predictor = model.deploy(accept_eula=accept_eula)

print ('Done! Move to the next cell ->')

Using model 'huggingface-llm-qwen2-5-coder-32b-instruct' with wildcard version identifier '*'. You can pin to version '1.0.6' for more stable results. Note that models may have different input/output signatures after a major version upgrade.


----------------------------------------------------*

Please check the troubleshooting guide for common errors: https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-python-sdk-troubleshooting.html#sagemaker-python-sdk-troubleshooting-create-endpoint


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:12                                                                                   │
│                                                                                                  │
│    9                                                                                             │
│   10 # deploy the model to an endpoint                                                           │
│   11 # this will take a while                                                                    │
│ ❱ 12 predictor = model.deploy(accept_eula=accept_eula)                                           │
│   13                                                                                             │
│   14 print ('Done! Move to the next cell ->')                                                    │
│   15                                                                                             │
│                                                                                                  │
│ /Users/simondavies/venv-jumpstart-lab1/lib/python3.13/site-packages/sagemaker/jumpstart/model.py │
│ :844 in deploy                                                                                   │
│                                                                                                  │
│    841 │   │   )                                                                                 │
│    842 │   │                                                                                     │
│    843 │   │   try:                                                                              │
│ ❱  844 │   │   │   predictor = super(JumpStartModel, self).deploy(**deploy_kwargs.to_kwargs_dic  │
│    845 │   │   except ClientError as e:                                                          │
│    846 │   │   │   subscription_link = verify_model_region_and_return_specs(                     │
│    847 │   │   │   │   region=self.region,                                                       │
│                                                                                                  │
│ /Users/simondavies/venv-jumpstart-lab1/lib/python3.13/site-packages/sagemaker/model.py:1814 in   │
│ deploy                                                                                           │
│                                                                                                  │
│   1811 │   │   │   │   )                                                                         │
│   1812 │   │   │   │   self.sagemaker_session.update_endpoint(self.endpoint_name, endpoint_conf  │
│   1813 │   │   │   else:                                                                         │
│ ❱ 1814 │   │   │   │   self.sagemaker_session.endpoint_from_production_variants(                 │
│   1815 │   │   │   │   │   name=self.endpoint_name,                                              │
│   1816 │   │   │   │   │   production_variants=[production_variant],                             │
│   1817 │   │   │   │   │   tags=tags,                                                            │
│                                                                                                  │
│ /Users/simondavies/venv-jumpstart-lab1/lib/python3.13/site-packages/sagemaker/session.py:6250 in │
│ endpoint_from_production_variants                                                                │
│                                                                                                  │
│   6247 │   │   logger.info("Creating endpoint-config with name %s", name)                        │
│   6248 │   │   self.sagemaker_client.create_endpoint_config(**config_options)                    │
│   6249 │   │                                                                                     │
│ ❱ 6250 │   │   return self.create_endpoint(                

In [ ]:
# test the endpoint with some example payloads
example_payloads = model.retrieve_all_examples()

for payload in example_payloads:
    response = predictor.predict(payload.body)
    response = response[0] if isinstance(response, list) else response
    print("Input:\n", payload.body, end="\n\n")
    print("Output:\n", response["generated_text"].strip(), end="\n\n\n")

# <span style="color:DarkSeaGreen">Create Scalability Plan</span>

In [ ]:
# https://github.com/aws/amazon-sagemaker-examples/blob/main/inference/generativeai/huggingfacetgi/meta-llama/llama3-8b/faster-autoscaling/realtime-endpoints/FasterAutoscaling-SME-Llama3-8B-AppAutoScaling.ipynb



# <span style="color:DarkSeaGreen">Clean Up Architecture</span>
### Only do this if you have finished with this lab and you no longer need any of the architecture created!!!

In [20]:
# when finished with the endpoint, delete it
predictor.delete_predictor()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # when finished with the endpoint, delete it                                                 │
│ ❱ 2 predictor.delete_predictor()                                                                 │
│   3                                                                                              │
│                                                                                                  │
│ /Users/simondavies/venv-jumpstart-lab1/lib/python3.13/site-packages/sagemaker/base_predictor.py: │
│ 542 in delete_predictor                                                                          │
│                                                                                                  │
│   539 │   │   if self.component_name:                                                            │
│   540 │   │   │   self.sagemaker_session.delete_inference_component(self.component_name, wait=   │
│   541 │   │   else:                                                                              │
│ ❱ 542 │   │   │   self.delete_endpoint()                                                         │
│   543 │                                                                                          │
│   544 │   def update_predictor(                                                                  │
│   545 │   │   self,                                                                              │
│                                                                                                  │
│ /Users/simondavies/venv-jumpstart-lab1/lib/python3.13/site-packages/sagemaker/base_predictor.py: │
│ 527 in delete_endpoint                                                                           │
│                                                                                                  │
│   524 │   │   │   │   be deleted. If False, only endpoint will be deleted.                       │
│   525 │   │   """                                                                                │
│   526 │   │   if delete_endpoint_config:                                                         │
│ ❱ 527 │   │   │   self._delete_endpoint_config()                                                 │
│   528 │   │                                                                                      │
│   529 │   │   self.sagemaker_session.delete_endpoint(self.endpoint_name)                         │
│   530                                                                                            │
│                                                                                                  │
│ /Users/simondavies/venv-jumpstart-lab1/lib/python3.13/site-packages/sagemaker/base_predictor.py: │
│ 512 in _delete_endpoint_config                                                                   │
│                                                                                                  │
│   509 │   def _delete_endpoint_config(self):                                                     │
│   510 │   │   """Delete the Amazon SageMaker endpoint configuration"""                           │
│   511 │   │   current_endpoint_config_name = self._get_endpoint_config_name()                    │
│ ❱ 512 │   │   self.sagemaker_session.delete_endpoint_config(current_endpoint_config_name)        │
│   513 │                                                                                          │
│   514 │   def delete_endpoint(self, delete_endpoint_config=True):                                │
│   515 │   │   """Delete the Amazon SageMaker endpoint backing this predictor.                    │
│                                                            

In [21]:
# delete roles and policies
iam.detach_role_policy(
    RoleName=myRoleSageMakerExecution, PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)
iam.delete_role(RoleName=myRoleSageMakerExecution)

{'ResponseMetadata': {'RequestId': 'e9a8603d-6369-4db5-9b96-f7d5eb245aca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 19 Sep 2025 10:22:00 GMT',
   'x-amzn-requestid': 'e9a8603d-6369-4db5-9b96-f7d5eb245aca',
   'content-type': 'text/xml',
   'content-length': '200'},
  'RetryAttempts': 0}}

# <span style="color:DarkSeaGreen">Clean Up venv</span>
### Clean up as below if finished with this lab and running in VSCode or equivalent local IDE
#### Note these are macOS specific
- Run the commands of the cell below in a terminal window if you need to clean up a local venv
  - Note if you copy and paste the entire cell and run as one you will get zsh: command not found: # errors because of the comments, but you can ignore
  - Remember to restart the kernel to refresh whats available

In [ ]:
# deactivate the venv
deactivate 
# remove it and its contents if not needed
rm -rf venv-jumpstart-lab1 